In [1]:
import requests as rq
import lxml.html as lx
import pandas as pd
import re

In [2]:
#read html + make df for general stats


url = 'https://www.espn.com/nba/team/stats/_/name/sac/salary'
names = pd.read_html(url)[0]
stats = pd.read_html(url)[1]
kings_stats = pd.concat([names,stats],axis = "columns")

In [21]:
# read html + make df for shooting stats


shoot = pd.read_html(url)[3]
kings_shoot_stats = pd.concat([names,shoot], axis = "columns")
kings_shoot_stats

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,De'Aaron Fox PG,9.6,17.7,54.1,2.0,5.1,39.5,4.3,5.3,81.0,7.6,12.6,59.9,1.435,0.60
1,Domantas Sabonis PF,5.9,10.3,57.7,0.3,1.1,27.8,4.7,6.4,73.4,5.6,9.2,61.1,1.640,0.59
2,Kevin Huerter SG,5.6,11.3,50.0,3.6,7.4,49.6,1.5,2.3,66.7,2.0,3.9,50.7,1.458,0.66
3,Malik Monk SG,4.9,10.4,46.9,2.0,5.5,36.2,1.9,2.1,91.4,2.9,4.9,59.0,1.311,0.56
4,Harrison Barnes SF,4.1,8.6,47.3,1.0,3.5,28.8,4.1,4.7,86.3,3.1,5.1,59.8,1.534,0.53
5,Keegan Murray SF,4.0,9.7,41.4,1.9,5.7,32.9,1.0,1.2,83.3,2.1,4.0,53.3,1.124,0.51
6,Terence Davis SG,2.4,5.3,45.2,1.3,3.4,38.9,0.8,0.9,85.7,1.1,1.9,56.7,1.298,0.58
7,Trey Lyles PF,2.3,5.6,40.5,1.3,3.9,33.9,0.9,1.2,72.2,0.9,1.7,56.0,1.202,0.52
8,Davion Mitchell PG,2.6,5.8,44.9,0.8,2.7,30.4,0.2,0.4,57.1,1.8,3.1,57.7,1.082,0.52
9,Chimezie Metu PF,2.4,4.1,58.5,0.2,1.2,20.0,0.5,0.5,85.7,2.2,2.9,73.7,1.340,0.61


In [22]:
# clean up "name" and drop total from df


words = []
for i in kings_shoot_stats["Name"]:
    words.append(i.split()[0:2])
words = pd.DataFrame(words)
words.columns = ["First","Last"]
words["Names"] = words['First']+" "+ words["Last"]
words = words.drop("First", axis = 1).drop("Last", axis = 1)
kings_shoot_stats["Name"] = words
kings_shoot_stats = kings_shoot_stats.drop(16)
kings_shoot_stats

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,De'Aaron Fox,9.6,17.7,54.1,2.0,5.1,39.5,4.3,5.3,81.0,7.6,12.6,59.9,1.435,0.60
1,Domantas Sabonis,5.9,10.3,57.7,0.3,1.1,27.8,4.7,6.4,73.4,5.6,9.2,61.1,1.640,0.59
2,Kevin Huerter,5.6,11.3,50.0,3.6,7.4,49.6,1.5,2.3,66.7,2.0,3.9,50.7,1.458,0.66
3,Malik Monk,4.9,10.4,46.9,2.0,5.5,36.2,1.9,2.1,91.4,2.9,4.9,59.0,1.311,0.56
4,Harrison Barnes,4.1,8.6,47.3,1.0,3.5,28.8,4.1,4.7,86.3,3.1,5.1,59.8,1.534,0.53
5,Keegan Murray,4.0,9.7,41.4,1.9,5.7,32.9,1.0,1.2,83.3,2.1,4.0,53.3,1.124,0.51
6,Terence Davis,2.4,5.3,45.2,1.3,3.4,38.9,0.8,0.9,85.7,1.1,1.9,56.7,1.298,0.58
7,Trey Lyles,2.3,5.6,40.5,1.3,3.9,33.9,0.9,1.2,72.2,0.9,1.7,56.0,1.202,0.52
8,Davion Mitchell,2.6,5.8,44.9,0.8,2.7,30.4,0.2,0.4,57.1,1.8,3.1,57.7,1.082,0.52
9,Chimezie Metu,2.4,4.1,58.5,0.2,1.2,20.0,0.5,0.5,85.7,2.2,2.9,73.7,1.340,0.61


In [5]:
#create salary df
url2 = "https://www.espn.com/nba/team/roster/_/name/sac/salary"
kings_salary_raw = pd.read_html(url2)[0]

In [6]:
#Strips player number from each players name

for i in range(0,len(kings_salary_raw)):
    kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


/tmp/ipykernel_2933/3064642318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


In [7]:
#Cleans dataframe to only player name + salary
kings_salary = kings_salary_raw.set_index("Name").drop('Unnamed: 0', axis = 1)
kings_salary.columns
kings_salary = kings_salary.drop(['POS', 'Age', 'HT', 'WT', 'College'], axis = 1)

In [8]:
#combine dfs. final kings 2022 df


kings_stats_salary = kings_shoot_stats.join(kings_salary, on = "Name")

In [9]:
nbateamstats = pd.DataFrame()
for i in range(2023,2018, -1):
    tempurl = "https://www.espn.com/nba/stats/team/_/view/opponent/season/"+str(i)+"/seasontype/2"
    team_names = pd.read_html(tempurl)[0]
    team_stats = pd.read_html(tempurl)[1]
    team_stats
    team_concat = pd.concat([team_names,team_stats],axis = "columns")
    team_concat['Team'] = str(i) + " " + team_concat['Team'].astype(str)
    nbateamstats = pd.concat([nbateamstats, team_concat], ignore_index = True)
nbateamstats



,RK,Team,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF
0,1,2023 Dallas Mavericks,17,106.5,38.6,81.4,47.5,10.5,28.6,36.8,...,23.2,80.5,9.1,32.8,41.8,22.9,6.1,5.1,15.1,23.2
1,2,2023 Philadelphia 76ers,18,106.6,39.6,84.7,46.8,10.1,31.9,31.5,...,22.6,76.6,9.6,33.7,43.3,24.2,6.4,5.0,14.5,19.5
2,3,2023 LA Clippers,19,106.8,39.6,88.9,44.6,12.2,35.9,34.0,...,20.5,74.8,10.9,32.5,43.4,23.5,8.5,4.1,13.3,19.3
3,4,2023 Milwaukee Bucks,17,107.5,40.2,89.7,44.8,11.4,31.8,35.9,...,20.1,78.4,9.8,33.5,43.2,21.8,6.5,5.5,13.6,20.8
4,5,2023 Cleveland Cavaliers,18,107.6,38.8,85.3,45.5,11.9,32.9,36.1,...,23.2,77.5,9.3,30.2,39.4,23.8,8.4,4.4,14.3,23.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,26,2019 Sacramento Kings,82,115.3,42.3,90.6,46.6,12.0,34.6,34.8,...,24.4,77.0,11.2,36.8,48.0,24.7,7.5,5.1,15.3,21.4
146,27,2019 New Orleans Pelicans,82,116.8,43.3,92.8,46.6,12.3,33.7,36.4,...,23.5,76.4,11.0,34.8,45.8,26.5,8.6,5.4,13.1,21.0
147,28,2019 Phoenix Suns,82,116.8,42.5,88.2,48.2,11.1,29.9,37.2,...,27.6,75.3,11.8,35.3,47.2,25.9,9.0,5.0,15.2,20.7
148,29,2019 Washington Wizards,82,116.9,43.4,90.5,48.0,12.1,32.8,37.0,...,23.1,77.8,11.4,35.8,47.3,26.2,7.7,4.6,15.0,21.2


In [10]:
#nbateams = tab_dict['tab_2021']["Team"]
nbateams = pd.read_html("https://www.espn.com/nba/stats/team/_/view/opponent/season/2023/seasontype/2")[0]["Team"]

In [11]:
def clean_teams(team, stats):
    team1 = team.columns[0]
    team.columns = ["Team"]
    new_row = pd.DataFrame({team.columns[0]:team1}, index=[0])
    team = pd.concat([new_row,team.loc[:]]).reset_index(drop=True)
    for i in range(0, len(team.iloc[:,0])):
        for teamname in nbateams:
            if team.iloc[:,0][i].__contains__(teamname):
                team.iloc[:,0][i] = teamname
                break
            else:
                continue
    result = team.join(stats.iloc[:,0:2])
    return(result)
    
def add_playoff_status(df, year):
    if year == 2023:
        df["Made Playoffs"] = None
        return(df)
    df["Made Playoffs"] = 0
    if year in [2019, 2020]:
        for index in df.index:
            if index < 8:
                df.iloc[index,3] = 1
    else:
        for index in df.index:
            if index < 10:
                df.iloc[index,3] = 1
    return(df)


league_wins_losses = pd.DataFrame()
for i in range(2023,2018, -1):
    urlstandings = "https://www.espn.com/nba/standings/_/season/"+ str(i)
    teams_east = pd.read_html(urlstandings)[0]
    stats_east = pd.read_html(urlstandings)[1]
    teams_west = pd.read_html(urlstandings)[2]
    stats_west = pd.read_html(urlstandings)[3]
    teams_east = clean_teams(teams_east, stats_east)
    teams_east = add_playoff_status(teams_east, i)
    teams_west = clean_teams(teams_west, stats_west)
    teams_west = add_playoff_status(teams_west, i)
    df_win_loss = pd.concat([teams_east, teams_west], ignore_index = True)
    df_win_loss['Team'] = str(i) + " " + df_win_loss['Team'].astype(str)
    league_wins_losses = pd.concat([league_wins_losses, df_win_loss], ignore_index = True)
league_wins_losses
    

,Team,W,L,Made Playoffs
0,2023 Boston Celtics,14,4,None
1,2023 Milwaukee Bucks,12,5,None
2,2023 Cleveland Cavaliers,12,6,None
3,2023 Atlanta Hawks,11,7,None
4,2023 Indiana Pacers,10,7,None
...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0
146,2019 Memphis Grizzlies,33,49,0
147,2019 New Orleans Pelicans,33,49,0
148,2019 Dallas Mavericks,33,49,0


In [12]:
fullnbastats = pd.merge(league_wins_losses, nbateamstats, on="Team")
fullnbastats = fullnbastats.drop(['RK'], axis=1)
fullnbastats

,Team,W,L,Made Playoffs,GP,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF
0,2023 Boston Celtics,14,4,None,18,113.4,42.3,90.6,46.7,11.6,...,21.8,78.6,9.9,34.5,44.4,22.2,6.1,3.8,11.8,20.6
1,2023 Milwaukee Bucks,12,5,None,17,107.5,40.2,89.7,44.8,11.4,...,20.1,78.4,9.8,33.5,43.2,21.8,6.5,5.5,13.6,20.8
2,2023 Cleveland Cavaliers,12,6,None,18,107.6,38.8,85.3,45.5,11.9,...,23.2,77.5,9.3,30.2,39.4,23.8,8.4,4.4,14.3,23.6
3,2023 Atlanta Hawks,11,7,None,18,113.9,41.4,89.3,46.3,10.7,...,26.1,78.5,10.9,35.2,46.2,23.3,6.7,5.7,14.3,21.7
4,2023 Indiana Pacers,10,7,None,17,114.1,40.1,86.8,46.2,12.6,...,25.9,82.0,10.7,32.5,43.2,24.8,8.3,5.7,15.0,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0,82,114.0,42.1,90.1,46.7,12.7,...,22.3,76.7,11.2,34.4,45.6,26.2,6.6,5.5,14.3,21.9
146,2019 Memphis Grizzlies,33,49,0,82,106.1,37.6,83.4,45.1,11.6,...,24.7,78.4,9.5,35.2,44.7,23.4,7.6,4.9,14.5,21.4
147,2019 New Orleans Pelicans,33,49,0,82,116.8,43.3,92.8,46.6,12.3,...,23.5,76.4,11.0,34.8,45.8,26.5,8.6,5.4,13.1,21.0
148,2019 Dallas Mavericks,33,49,0,82,110.1,40.9,89.2,45.9,11.2,...,22.0,77.7,10.2,34.6,44.8,24.5,7.8,4.5,12.5,23.2


In [13]:
nbaefficiencystats = pd.DataFrame()
for year in range(2023,2018, -1):
    efficiencyurl = "http://www.espn.com/nba/hollinger/teamstats/_/sort/defensiveEff/year/"+ str(year)
    efficiencystats = pd.read_html(efficiencyurl)[0].iloc[1: , 1:]
    efficiencystats = pd.DataFrame(efficiencystats.values[1:], columns=efficiencystats.iloc[0])
    for i in range(0, len(efficiencystats.iloc[:,0])):
        for teamname in nbateams:
            if efficiencystats.iloc[:,0][i] == "LA Lakers":
                efficiencystats.iloc[:,0][i] = "Los Angeles Lakers"
                break
            if efficiencystats.iloc[:,0][i] in teamname:
                efficiencystats.iloc[:,0][i] = teamname
                break
            else:
                continue
    efficiencystats = efficiencystats.rename(columns = {"TEAM": "Team"})
    efficiencystats['Team'] = str(year) + " " + efficiencystats['Team'].astype(str)
    efficiencystats = efficiencystats[["Team", "PACE", "EFF FG%", "OFF EFF", "DEF EFF"]]
    nbaefficiencystats = pd.concat([nbaefficiencystats, efficiencystats], ignore_index = True)
nbaefficiencystats

1,Team,PACE,EFF FG%,OFF EFF,DEF EFF
0,2023 Milwaukee Bucks,102.4,52.0,106.7,104.7
1,2023 LA Clippers,100.6,54.7,105.8,105.8
2,2023 Philadelphia 76ers,99.9,54.1,109.2,106.2
3,2023 Cleveland Cavaliers,98.6,56.0,113.6,106.3
4,2023 New Orleans Pelicans,101.8,55.4,113.8,106.6
...,...,...,...,...,...
145,2019 Chicago Bulls,101.1,50.5,102.5,110.9
146,2019 Atlanta Hawks,106.4,52.2,105.5,111.2
147,2019 Washington Wizards,103.7,53.1,108.6,111.3
148,2019 Phoenix Suns,102.9,51.4,103.6,112.2


In [14]:
fullnbastats = pd.merge(fullnbastats, nbaefficiencystats, on="Team")
fullnbastats

,Team,W,L,Made Playoffs,GP,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF
0,2023 Boston Celtics,14,4,None,18,113.4,42.3,90.6,46.7,11.6,...,44.4,22.2,6.1,3.8,11.8,20.6,101.2,58.2,117.4,110.3
1,2023 Milwaukee Bucks,12,5,None,17,107.5,40.2,89.7,44.8,11.4,...,43.2,21.8,6.5,5.5,13.6,20.8,102.4,52.0,106.7,104.7
2,2023 Cleveland Cavaliers,12,6,None,18,107.6,38.8,85.3,45.5,11.9,...,39.4,23.8,8.4,4.4,14.3,23.6,98.6,56.0,113.6,106.3
3,2023 Atlanta Hawks,11,7,None,18,113.9,41.4,89.3,46.3,10.7,...,46.2,23.3,6.7,5.7,14.3,21.7,103.9,51.2,108.7,108.5
4,2023 Indiana Pacers,10,7,None,17,114.1,40.1,86.8,46.2,12.6,...,43.2,24.8,8.3,5.7,15.0,20.5,104.1,54.1,110.3,110.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0,82,114.0,42.1,90.1,46.7,12.7,...,45.6,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2
146,2019 Memphis Grizzlies,33,49,0,82,106.1,37.6,83.4,45.1,11.6,...,44.7,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2
147,2019 New Orleans Pelicans,33,49,0,82,116.8,43.3,92.8,46.6,12.3,...,45.8,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4
148,2019 Dallas Mavericks,33,49,0,82,110.1,40.9,89.2,45.9,11.2,...,44.8,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3


In [15]:
playoffstat = fullnbastats.pop("Made Playoffs")
fullnbastats["Made Playoffs"] = playoffstat
fullnbastats

,Team,W,L,GP,PTS,FGM,FGA,FG%,3PM,3PA,...,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF,Made Playoffs
0,2023 Boston Celtics,14,4,18,113.4,42.3,90.6,46.7,11.6,32.1,...,22.2,6.1,3.8,11.8,20.6,101.2,58.2,117.4,110.3,None
1,2023 Milwaukee Bucks,12,5,17,107.5,40.2,89.7,44.8,11.4,31.8,...,21.8,6.5,5.5,13.6,20.8,102.4,52.0,106.7,104.7,None
2,2023 Cleveland Cavaliers,12,6,18,107.6,38.8,85.3,45.5,11.9,32.9,...,23.8,8.4,4.4,14.3,23.6,98.6,56.0,113.6,106.3,None
3,2023 Atlanta Hawks,11,7,18,113.9,41.4,89.3,46.3,10.7,32.3,...,23.3,6.7,5.7,14.3,21.7,103.9,51.2,108.7,108.5,None
4,2023 Indiana Pacers,10,7,17,114.1,40.1,86.8,46.2,12.6,33.6,...,24.8,8.3,5.7,15.0,20.5,104.1,54.1,110.3,110.7,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,82,114.0,42.1,90.1,46.7,12.7,33.6,...,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2,0
146,2019 Memphis Grizzlies,33,49,82,106.1,37.6,83.4,45.1,11.6,32.2,...,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2,0
147,2019 New Orleans Pelicans,33,49,82,116.8,43.3,92.8,46.6,12.3,33.7,...,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4,0
148,2019 Dallas Mavericks,33,49,82,110.1,40.9,89.2,45.9,11.2,32.5,...,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3,0


In [16]:
kings_stats["Name"] = kings_stats_salary["Name"]
kings_stats

,Name,GP,GS,MIN,PTS,OR,DR,REB,AST,STL,BLK,TO,PF,AST/TO,PER
0,De'Aaron Fox,16,16.0,32.4,25.4,0.6,4.4,5.0,6.3,1.3,0.4,2.7,2.9,2.3,24.85
1,Domantas Sabonis,17,17.0,32.3,16.9,2.2,8.9,11.2,6.2,0.8,0.4,2.4,3.9,2.6,21.80
2,Kevin Huerter,17,17.0,32.2,16.5,0.4,2.8,3.2,3.4,1.1,0.5,1.6,2.5,2.1,16.17
3,Malik Monk,17,0.0,22.1,13.6,0.4,1.8,2.2,4.1,0.6,0.2,1.8,1.9,2.3,17.98
4,Harrison Barnes,17,17.0,31.4,13.2,1.1,4.6,5.7,2.0,0.8,0.2,1.2,1.3,1.7,14.27
5,Keegan Murray,15,13.0,29.5,10.9,0.7,2.9,3.5,1.1,0.8,0.6,1.5,2.1,0.7,8.38
6,Terence Davis,16,1.0,12.6,6.8,0.4,2.2,2.6,1.4,0.8,0.2,0.6,1.8,2.3,18.22
7,Trey Lyles,15,0.0,15.1,6.7,1.1,1.8,2.9,0.8,0.4,0.5,1.0,1.2,0.8,12.34
8,Davion Mitchell,17,1.0,18.9,6.2,0.1,1.1,1.2,2.2,0.4,0.1,1.1,1.5,1.9,7.48
9,Chimezie Metu,13,0.0,12.4,5.5,0.5,2.7,3.2,0.8,0.4,0.1,0.6,1.2,1.3,15.27


In [17]:
pd.set_option('display.max_columns', None)
kings_player_shooting_salary = pd.merge(kings_stats_salary,kings_stats, how = "left", on = "Name")
salary = kings_player_shooting_salary.pop("Salary")
kings_player_shooting_salary = pd.concat([kings_player_shooting_salary, salary], axis = 'columns')

In [18]:
kings_player_shooting_salary

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF,GP,GS,MIN,PTS,OR,DR,REB,AST,STL,BLK,TO,PF,AST/TO,PER,Salary
0,De'Aaron Fox,9.6,17.7,54.1,2.0,5.1,39.5,4.3,5.3,81.0,7.6,12.6,59.9,1.435,0.60,16,16.0,32.4,25.4,0.6,4.4,5.0,6.3,1.3,0.4,2.7,2.9,2.3,24.85,"$28,103,550"
1,Domantas Sabonis,5.9,10.3,57.7,0.3,1.1,27.8,4.7,6.4,73.4,5.6,9.2,61.1,1.640,0.59,17,17.0,32.3,16.9,2.2,8.9,11.2,6.2,0.8,0.4,2.4,3.9,2.6,21.80,"$18,500,000"
2,Kevin Huerter,5.6,11.3,50.0,3.6,7.4,49.6,1.5,2.3,66.7,2.0,3.9,50.7,1.458,0.66,17,17.0,32.2,16.5,0.4,2.8,3.2,3.4,1.1,0.5,1.6,2.5,2.1,16.17,"$14,508,929"
3,Malik Monk,4.9,10.4,46.9,2.0,5.5,36.2,1.9,2.1,91.4,2.9,4.9,59.0,1.311,0.56,17,0.0,22.1,13.6,0.4,1.8,2.2,4.1,0.6,0.2,1.8,1.9,2.3,17.98,"$9,472,219"
4,Harrison Barnes,4.1,8.6,47.3,1.0,3.5,28.8,4.1,4.7,86.3,3.1,5.1,59.8,1.534,0.53,17,17.0,31.4,13.2,1.1,4.6,5.7,2.0,0.8,0.2,1.2,1.3,1.7,14.27,"$18,352,273"
5,Keegan Murray,4.0,9.7,41.4,1.9,5.7,32.9,1.0,1.2,83.3,2.1,4.0,53.3,1.124,0.51,15,13.0,29.5,10.9,0.7,2.9,3.5,1.1,0.8,0.6,1.5,2.1,0.7,8.38,"$8,008,440"
6,Terence Davis,2.4,5.3,45.2,1.3,3.4,38.9,0.8,0.9,85.7,1.1,1.9,56.7,1.298,0.58,16,1.0,12.6,6.8,0.4,2.2,2.6,1.4,0.8,0.2,0.6,1.8,2.3,18.22,"$4,000,000"
7,Trey Lyles,2.3,5.6,40.5,1.3,3.9,33.9,0.9,1.2,72.2,0.9,1.7,56.0,1.202,0.52,15,0.0,15.1,6.7,1.1,1.8,2.9,0.8,0.4,0.5,1.0,1.2,0.8,12.34,"$2,700,000"
8,Davion Mitchell,2.6,5.8,44.9,0.8,2.7,30.4,0.2,0.4,57.1,1.8,3.1,57.7,1.082,0.52,17,1.0,18.9,6.2,0.1,1.1,1.2,2.2,0.4,0.1,1.1,1.5,1.9,7.48,"$4,833,600"
9,Chimezie Metu,2.4,4.1,58.5,0.2,1.2,20.0,0.5,0.5,85.7,2.2,2.9,73.7,1.340,0.61,13,0.0,12.4,5.5,0.5,2.7,3.2,0.8,0.4,0.1,0.6,1.2,1.3,15.27,"$1,910,680"


In [19]:
#Dataframes available:
kings_stats
kings_shoot_stats
kings_salary
nbateams
kings_player_shooting_salary
league_wins_losses
nbateamstats
nbaefficiencystats
fullnbastats

,Team,W,L,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF,Made Playoffs
0,2023 Boston Celtics,14,4,18,113.4,42.3,90.6,46.7,11.6,32.1,36.0,17.2,21.8,78.6,9.9,34.5,44.4,22.2,6.1,3.8,11.8,20.6,101.2,58.2,117.4,110.3,None
1,2023 Milwaukee Bucks,12,5,17,107.5,40.2,89.7,44.8,11.4,31.8,35.9,15.8,20.1,78.4,9.8,33.5,43.2,21.8,6.5,5.5,13.6,20.8,102.4,52.0,106.7,104.7,None
2,2023 Cleveland Cavaliers,12,6,18,107.6,38.8,85.3,45.5,11.9,32.9,36.1,18.0,23.2,77.5,9.3,30.2,39.4,23.8,8.4,4.4,14.3,23.6,98.6,56.0,113.6,106.3,None
3,2023 Atlanta Hawks,11,7,18,113.9,41.4,89.3,46.3,10.7,32.3,33.0,20.5,26.1,78.5,10.9,35.2,46.2,23.3,6.7,5.7,14.3,21.7,103.9,51.2,108.7,108.5,None
4,2023 Indiana Pacers,10,7,17,114.1,40.1,86.8,46.2,12.6,33.6,37.6,21.2,25.9,82.0,10.7,32.5,43.2,24.8,8.3,5.7,15.0,20.5,104.1,54.1,110.3,110.7,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,82,114.0,42.1,90.1,46.7,12.7,33.6,37.8,17.1,22.3,76.7,11.2,34.4,45.6,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2,0
146,2019 Memphis Grizzlies,33,49,82,106.1,37.6,83.4,45.1,11.6,32.2,36.0,19.3,24.7,78.4,9.5,35.2,44.7,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2,0
147,2019 New Orleans Pelicans,33,49,82,116.8,43.3,92.8,46.6,12.3,33.7,36.4,17.9,23.5,76.4,11.0,34.8,45.8,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4,0
148,2019 Dallas Mavericks,33,49,82,110.1,40.9,89.2,45.9,11.2,32.5,34.4,17.1,22.0,77.7,10.2,34.6,44.8,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3,0


In [20]:
%store kings_player_shooting_salary
del kings_player_shooting_salary

Stored 'kings_player_shooting_salary' (DataFrame)
